<a id="2"></a>

<h1 style="
    background-color:#488286; 
    background-size: cover;
    background-position: center;
    font-family: 'Times New Roman', serif;
    font-size: 1.1em;
    color: white;
    text-align: center;
    padding: 15px;
    border-radius: 15px 50px;
    border: 1px solid black;
">
    Spam Ham Classification Project Using BOW And TFIDF And ML
</h1>

In [ ]:
import re
import gensim
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from gensim.utils import simple_preprocess
from tqdm import tqdm

In [ ]:
messages = pd.read_csv(
    r"data\SMSSpamCollection",
    sep="\t",
    names=["label", "message"],
)
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


#### Preprocessing

In [ ]:
snow_stem = SnowballStemmer('english')

corpus = []
for i in range(len(messages)):
    review = re.sub("[^a-zA-Z]", " ", messages["message"][i])
    review = review.lower()
    review = review.split()
    review = [
        snow_stem.stem(word)
        for word in review
        if not word in stopwords.words("english")
    ]
    review = " ".join(review)
    corpus.append(review)

corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkts st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breath

#### Create Bag Of Words

In [ ]:
## Feature Selection
y = pd.get_dummies(messages['label'])
y

,ham,spam
0,True,False
1,True,False
2,False,True
3,True,False
4,True,False
...,...,...
5567,False,True
5568,True,False
5569,True,False
5570,True,False


In [ ]:
y = y.iloc[:, 0].values
print(y)


[ True  True False ...  True  True  True]


In [ ]:
## Train Test Split
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.20)

In [ ]:
cv = CountVectorizer(max_features=2500, ngram_range=(1, 2))
X_train = cv.fit_transform(X_train).toarray()
X_test = cv.transform(X_test).toarray()

In [ ]:
cv.vocabulary_

{'think': 2106,
 'first': 708,
 'time': 2128,
 'saw': 1795,
 'class': 360,
 'first time': 709,
 'someon': 1922,
 'know': 1086,
 'ask': 101,
 'date': 497,
 'servic': 1846,
 'contact': 417,
 'cant': 286,
 'guess': 884,
 'call': 244,
 'reveal': 1747,
 'pobox': 1600,
 'xy': 2468,
 'someon know': 1924,
 'know ask': 1087,
 'ask date': 102,
 'date servic': 498,
 'servic contact': 1848,
 'contact cant': 419,
 'cant guess': 288,
 'guess call': 885,
 'call reveal': 264,
 'convey': 426,
 'regard': 1714,
 'convey regard': 428,
 'ran': 1680,
 'man': 1246,
 'make': 1237,
 'pretti': 1634,
 'babi': 131,
 'togeth': 2153,
 'tot': 2169,
 'din': 543,
 'found': 728,
 'group': 863,
 'liao': 1147,
 'network': 1401,
 'oper': 1489,
 'free': 731,
 'visit': 2308,
 'biz': 180,
 'network oper': 1403,
 'ha': 889,
 'say': 1796,
 'read': 1693,
 'anyth': 74,
 'way': 2359,
 'seem': 1821,
 'like': 1158,
 'save': 1794,
 'friday': 747,
 'pub': 1656,
 'dunno': 591,
 'lei': 1136,
 'might': 1293,
 'eatin': 604,
 'wif': 2401,

In [ ]:
spamDetectorModel = MultinomialNB().fit(X_train,y_train)

In [ ]:
y_pred = spamDetectorModel.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.97      0.90      0.93       154
        True       0.98      1.00      0.99       961

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



### Creating The TF-IDF Model

In [ ]:
## Train Test Split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.20)

In [ ]:
tf = TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X_train= tf.fit_transform(X_train).toarray()
X_test= tf.transform(X_test).toarray()

In [ ]:
spamDetectorModel_tf = MultinomialNB().fit(X_train, y_train)

In [ ]:
y_pred = spamDetectorModel_tf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       1.00      0.86      0.92       149
        True       0.98      1.00      0.99       966

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115



<a id="2"></a>

<h1 style="
    background-color:#488286; 
    background-size: cover;
    background-position: center;
    font-family: 'Times New Roman', serif;
    font-size: 1.1em;
    color: white;
    text-align: center;
    padding: 15px;
    border-radius: 15px 50px;
    border: 1px solid black;
">
   Using Word2vec , AvgWord2vec
</h1>

#### Preprocessing

In [ ]:
lem = WordNetLemmatizer()

corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
## Find na values in message
[
    [i, j, k]
    for i, j, k in zip(list(map(len, corpus)), corpus, messages["message"])
    if i < 1
]

[[0, '', '645'], [0, '', ':) '], [0, '', ':-) :-)']]

In [ ]:
words = []

for sentence in corpus :
    sentence_token = sent_tokenize(sentence)
    for sentence in sentence_token:
        words.append(simple_preprocess(sentence))

In [ ]:
words

[['go',
  'until',
  'jurong',
  'point',
  'crazy',
  'available',
  'only',
  'in',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'there',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'in',
  'wkly',
  'comp',
  'to',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'to',
  'to',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  'over'],
 ['dun', 'say', 'so', 'early', 'hor', 'already', 'then', 'say'],
 ['nah',
  'don',
  'think',
  'he',
  'go',
  'to',
  'usf',
  'he',
  'life',
  'around',
  'here',
  'though'],
 ['freemsg',
  'hey',
  'there',
  'darling',
  'it',
  'been',
  'week',
  'now',
  'and',
  'no',
  'word',
  'back',
  'like',
  'some',
  'fun',
  'you',
  'up',
  'for',
  'it',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'to',
  'send',
  'to',
  'rcv'],
 ['even',
  'my',
  'brother',
  'is',
  'not',
  'like',
  'to',
  'spea

In [ ]:
model = gensim.models.Word2Vec(words)

In [ ]:
print(model.wv['good'].shape)
model.wv.similar_by_word('good')

(100,)


[('not', 0.9985684156417847),
 ('night', 0.9985409379005432),
 ('morning', 0.9984925985336304),
 ('wa', 0.9984391331672668),
 ('where', 0.9984146952629089),
 ('all', 0.9984033107757568),
 ('oh', 0.9983351826667786),
 ('well', 0.9982500672340393),
 ('there', 0.9982338547706604),
 ('about', 0.9982236623764038)]

In [ ]:
def avg_word2vec(doc):
    return np.mean(
        [model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0
    )

X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|          | 0/5569 [00:00<?, ?it/s]c:\Users\OnlyGod\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\OnlyGod\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5569/5569 [00:00<00:00, 13086.60it/s]


In [ ]:
X

[array([-0.17375763,  0.24355207,  0.11332963,  0.10552812,  0.09658041,
        -0.48821822,  0.1741454 ,  0.46849063, -0.28487685, -0.12638088,
        -0.15549706, -0.36545995, -0.05022654,  0.12282788,  0.18181235,
        -0.15042625,  0.13528977, -0.2926174 , -0.07471685, -0.5081456 ,
         0.20550394,  0.1362118 ,  0.07550078, -0.2241738 , -0.02761667,
        -0.01604725, -0.21632586, -0.18318073, -0.25023344,  0.03273183,
         0.3052045 ,  0.02022651,  0.10673884, -0.18275684, -0.13327919,
         0.38664564,  0.07371175, -0.12709738, -0.10964738, -0.4771162 ,
         0.1271727 , -0.25376898, -0.17811204,  0.01763064,  0.14085612,
         0.00922515, -0.12558143, -0.03288528,  0.19708076,  0.13812533,
         0.169322  , -0.19188128, -0.0649887 ,  0.06702939, -0.07082278,
         0.04178318,  0.13444136, -0.0018082 , -0.36485514,  0.17988402,
        -0.00304421,  0.15569806,  0.0090104 , -0.11178939, -0.29132894,
         0.29447567,  0.10293224,  0.22277826, -0.3

In [ ]:
## Dependent Features
## Output Features
y = messages[list(map(lambda x:len(x)>0,corpus))]
y = pd.get_dummies(y['label'])
y = y.iloc[:,0].values.astype(int)

In [ ]:
shapes = [x.shape for x in X]
from collections import Counter
print(Counter(shapes))


Counter({(100,): 5557, (): 12})


In [ ]:
correct_dim = 100  

X_fixed = []
for x in X:
    if x.shape == (correct_dim,):
        X_fixed.append(x)
    else:
        X_fixed.append(np.zeros(correct_dim))


In [ ]:
df = pd.DataFrame(np.vstack(X_fixed))
df['Output']=y
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
0,-0.173758,0.243552,0.113330,0.105528,0.096580,-0.488218,0.174145,0.468491,-0.284877,-0.126381,...,0.153625,0.035564,0.055462,0.421426,0.181880,0.141388,-0.203043,0.142126,0.003942,1
1,-0.161793,0.209952,0.095983,0.089363,0.091858,-0.421957,0.137961,0.411061,-0.247223,-0.102573,...,0.125318,0.025929,0.042963,0.354831,0.154923,0.124973,-0.185906,0.129499,-0.002928,1
2,-0.186305,0.259674,0.121032,0.125617,0.081436,-0.522198,0.175203,0.464734,-0.300483,-0.150525,...,0.156391,0.034562,0.042532,0.432179,0.168135,0.093744,-0.234619,0.170664,0.020154,0
3,-0.239057,0.329808,0.149676,0.141877,0.131973,-0.663537,0.230097,0.643719,-0.391433,-0.165444,...,0.204631,0.044253,0.082983,0.562791,0.254691,0.210340,-0.276623,0.192207,-0.005669,1
4,-0.202991,0.272336,0.132934,0.115575,0.117703,-0.560473,0.194442,0.546543,-0.333575,-0.146340,...,0.174343,0.042536,0.073420,0.481669,0.217221,0.174165,-0.239532,0.157350,-0.007331,1


In [ ]:
## Feature Selection
X = df.iloc[:,-1].to_frame()
y = df['Output'] 

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.20)

In [ ]:
print(X_train.shape)
print(y_train.shape)

(4455, 1)
(4455,)


In [ ]:
model = RandomForestClassifier().fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       155
           1       1.00      1.00      1.00       959

    accuracy                           1.00      1114
   macro avg       1.00      1.00      1.00      1114
weighted avg       1.00      1.00      1.00      1114

